In [1]:
import numpy as np
import pandas as pd
import math
pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format

# Outline

###  Part A. Affordable  Rentals
 1. Bring in 2013 PUMS data
 2. Assign county_id to each puma
 3. Bring in and clean, reformat county AMIs data
 4. Add variables for cost limits for ELI, VLI, LI, MI, and HI by bedroom count for each county
 5. Add count dummies for each unit based on income and unit type
 

###  Part B. Affordable Owning
 1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees
 2. Add count dummies for each unit based on income and unit type

### Part C. Weighting Sample to get estimates
 1. Multiply each dummy by the the household weight variable
 2. Calculte standard errors for each estimate
 
### Part D. Aggregating at PUMA Level
 1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in and create second dataset
 2. Aggregate at PUMA level for each dataset and export

## Part A1. Bring in 2013 PUMS data

In [2]:
variable_types = {"BDSP":"float","RMSP":"str","MV":"float","WGTP":"float","TAXP":"float"}
columns = ["PUMA","RT","BDSP","BLD","RNTP","MRGP","SMP","TAXP","CONP","TEN","VACS","VALP","GRPIP","GRNTP","HINCP","MV","WGTP"]
pums_df=pd.read_csv("2013_pums.csv", delimiter=",",usecols=columns,dtype=variable_types)
pums_df

,RT,PUMA,WGTP,BDSP,BLD,CONP,MRGP,RNTP,SMP,TEN,VACS,VALP,GRNTP,GRPIP,HINCP,MV,TAXP
0,H,7301,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,H,3702,38.00,3.00,2.00,nan,3500.00,nan,nan,1.00,nan,600000.00,nan,nan,464800.00,6.00,64.00
2,H,8503,78.00,2.00,6.00,430.00,1800.00,nan,460.00,1.00,nan,370000.00,nan,nan,16000.00,2.00,52.00
3,H,300,233.00,1.00,5.00,nan,nan,680.00,nan,3.00,nan,nan,780.00,19.00,50000.00,3.00,nan
4,H,7901,446.00,2.00,1.00,nan,nan,nan,nan,nan,3.00,140000.00,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153584,H,3725,154.00,2.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1210.00,101.00,6000.00,5.00,nan
153585,H,7102,67.00,4.00,2.00,nan,1600.00,nan,600.00,1.00,nan,122000.00,nan,nan,22200.00,4.00,29.00
153586,H,10701,104.00,3.00,2.00,nan,nan,580.00,nan,3.00,nan,nan,640.00,10.00,74500.00,5.00,nan
153587,H,5903,357.00,4.00,2.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan


In [3]:
pums_df.rename(columns={"RT":"record_type","BDSP":"number_bedrooms",
    "BLD":"units_in_structure",
    "RNTP":"monthly_rent",
    "MRGP":"first_mortgage",
    "SMP":"second_mortgage",
    "CONP":"condo_fee",
    "TEN":"tenure",
    "VACS":"vacancy_status",
    "VALP":"property_value",
    "GRPIP":"gross_rent_pct_of_income",
    "GRNTP":"gross_rent",
    "HINCP":"hh_income",
    "TAXP":"prop_tax",
    "WGTP":"weight",
    "MV":"moved_in"}, inplace=True)
pums_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax
0,H,7301,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,H,3702,38.00,3.00,2.00,nan,3500.00,nan,nan,1.00,nan,600000.00,nan,nan,464800.00,6.00,64.00
2,H,8503,78.00,2.00,6.00,430.00,1800.00,nan,460.00,1.00,nan,370000.00,nan,nan,16000.00,2.00,52.00
3,H,300,233.00,1.00,5.00,nan,nan,680.00,nan,3.00,nan,nan,780.00,19.00,50000.00,3.00,nan
4,H,7901,446.00,2.00,1.00,nan,nan,nan,nan,nan,3.00,140000.00,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153584,H,3725,154.00,2.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1210.00,101.00,6000.00,5.00,nan
153585,H,7102,67.00,4.00,2.00,nan,1600.00,nan,600.00,1.00,nan,122000.00,nan,nan,22200.00,4.00,29.00
153586,H,10701,104.00,3.00,2.00,nan,nan,580.00,nan,3.00,nan,nan,640.00,10.00,74500.00,5.00,nan
153587,H,5903,357.00,4.00,2.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan


## Part A2. Assign county id to each puma

In [4]:
# load in crosswalk file
crosswalk_df=pd.read_csv("C:/Users/Terner GSR/Box/Cost of Doing Nothing/Climate Impacts/Data/PUMS/PUMA_County_Crosswalk_v2.csv", delimiter=",")
crosswalk_df

,PUMA,county1,county2,county3,county4,county5,county6,county7
0,101,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
1,102,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
2,103,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
3,104,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
4,105,Alameda CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
260,11103,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
261,11104,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
262,11105,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN
263,11106,Ventura CA,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# add county name column to puma file
puma_county_df=pums_df.merge(crosswalk_df, how='left', left_on = "PUMA",right_on = "PUMA")
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7
0,H,7301,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN
1,H,3702,38.00,3.00,2.00,nan,3500.00,nan,nan,1.00,nan,600000.00,nan,nan,464800.00,6.00,64.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
2,H,8503,78.00,2.00,6.00,430.00,1800.00,nan,460.00,1.00,nan,370000.00,nan,nan,16000.00,2.00,52.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN
3,H,300,233.00,1.00,5.00,nan,nan,680.00,nan,3.00,nan,nan,780.00,19.00,50000.00,3.00,nan,Alpine CA,Amador CA,Calaveras CA,Inyo CA,Mariposa CA,Mono CA,Tuolumne CA
4,H,7901,446.00,2.00,1.00,nan,nan,nan,nan,nan,3.00,140000.00,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153584,H,3725,154.00,2.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1210.00,101.00,6000.00,5.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN
153585,H,7102,67.00,4.00,2.00,nan,1600.00,nan,600.00,1.00,nan,122000.00,nan,nan,22200.00,4.00,29.00,San Bernardino CA,NaN,NaN,NaN,NaN,NaN,NaN
153586,H,10701,104.00,3.00,2.00,nan,nan,580.00,nan,3.00,nan,nan,640.00,10.00,74500.00,5.00,nan,Tulare CA,NaN,NaN,NaN,NaN,NaN,NaN
153587,H,5903,357.00,4.00,2.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# remove " CA" from end of each county name
puma_county_df['County'] = puma_county_df['county1'].str.replace(r' CA', '')
puma_county_df['county2'] = puma_county_df['county2'].str.replace(r' CA', '')
puma_county_df['county3'] = puma_county_df['county3'].str.replace(r' CA', '')
puma_county_df['county4'] = puma_county_df['county4'].str.replace(r' CA', '')
puma_county_df['county5'] = puma_county_df['county5'].str.replace(r' CA', '')
puma_county_df['county6'] = puma_county_df['county6'].str.replace(r' CA', '')
puma_county_df['county7'] = puma_county_df['county7'].str.replace(r' CA', '')
puma_county_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County
0,H,7301,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego
1,H,3702,38.00,3.00,2.00,nan,3500.00,nan,nan,1.00,nan,600000.00,nan,nan,464800.00,6.00,64.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
2,H,8503,78.00,2.00,6.00,430.00,1800.00,nan,460.00,1.00,nan,370000.00,nan,nan,16000.00,2.00,52.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara
3,H,300,233.00,1.00,5.00,nan,nan,680.00,nan,3.00,nan,nan,780.00,19.00,50000.00,3.00,nan,Alpine CA,Amador,Calaveras,Inyo,Mariposa,Mono,Tuolumne,Alpine
4,H,7901,446.00,2.00,1.00,nan,nan,nan,nan,nan,3.00,140000.00,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153584,H,3725,154.00,2.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1210.00,101.00,6000.00,5.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles
153585,H,7102,67.00,4.00,2.00,nan,1600.00,nan,600.00,1.00,nan,122000.00,nan,nan,22200.00,4.00,29.00,San Bernardino CA,NaN,NaN,NaN,NaN,NaN,NaN,San Bernardino
153586,H,10701,104.00,3.00,2.00,nan,nan,580.00,nan,3.00,nan,nan,640.00,10.00,74500.00,5.00,nan,Tulare CA,NaN,NaN,NaN,NaN,NaN,NaN,Tulare
153587,H,5903,357.00,4.00,2.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange


## Part A3. Bring in and clean, reformat county AMIs data

In [7]:
#bring in 2018 county AMIs file
columns=["County","Income_Category","1","2","4","6","8"]
df = pd.read_csv("2013_amis.csv", delimiter=",", usecols=columns)
df

,County,Income_Category,1,2,4,6,8
0,Alameda County\n4-Per,Extremely Low,19650,22450,28050,32550,37050
1,Alameda County\n4-Per,Very Low Income,32750,37400,46750,54250,61750
2,Alameda County\n4-Per,Lower Income,46350,53000,66250,76850,87450
3,Alameda County\n4-Per,Median Income,65450,74800,93500,108450,123400
4,Alameda County\n4-Per,Moderate Income,78550,89750,112200,130150,148100
...,...,...,...,...,...,...,...
285,Yuba County\n4-Person,Extremely Low,12500,14250,17800,20650,23500
286,Yuba County\n4-Person,Very Low Income,20800,23800,29700,34500,39250
287,Yuba County\n4-Person,Lower Income,33250,38000,47500,55100,62700
288,Yuba County\n4-Person,Median Income,41600,47500,59400,68900,78400


In [8]:
#clean county names variable
df['County'] = df['County'].str.replace(r' County\n4-Person', '')
df['County'] = df['County'].str.replace(r' County\n4-Perso', '')
df['County'] = df['County'].str.replace(r' County\n4-Pers', '')
df['County'] = df['County'].str.replace(r' County\n4-Per', '')
df['County'] = df['County'].str.replace(r' County\n4-Pe', '')
df['County'] = df['County'].str.replace(r' County\n4-P', '')
df['County'] = df['County'].str.replace(r' County\n4-', '')
df['County'] = df['County'].str.replace(r' County\n4', '')
df['County'] = df['County'].str.replace(r' County\n', '')
df['County'] = df['County'].str.replace(r' County\ ' , '')
df['County'] = df['County'].str.replace(r' County', '')
df['County'] = df['County'].str.replace(r' Count', '')
df['County'] = df['County'].str.replace(r' Coun', '')
df

,County,Income_Category,1,2,4,6,8
0,Alameda,Extremely Low,19650,22450,28050,32550,37050
1,Alameda,Very Low Income,32750,37400,46750,54250,61750
2,Alameda,Lower Income,46350,53000,66250,76850,87450
3,Alameda,Median Income,65450,74800,93500,108450,123400
4,Alameda,Moderate Income,78550,89750,112200,130150,148100
...,...,...,...,...,...,...,...
285,Yuba,Extremely Low,12500,14250,17800,20650,23500
286,Yuba,Very Low Income,20800,23800,29700,34500,39250
287,Yuba,Lower Income,33250,38000,47500,55100,62700
288,Yuba,Median Income,41600,47500,59400,68900,78400


In [9]:
#create df for each income level and calculate max rents, rename variables
eli_df = df[df.Income_Category =="Extremely Low"].copy()
eli_df["1"]=eli_df["1"]*0.025
eli_df["2"]=eli_df["2"]*0.025
eli_df["4"]=eli_df["4"]*0.025
eli_df["6"]=eli_df["6"]*0.025
eli_df["8"]=eli_df["8"]*0.025
eli_df.rename(columns={"1":"ELI_studio","2":"ELI_1_br","4":"ELI_2_br","6":"ELI_3_br","8":"ELI_4_br"}, inplace=True)
eli_df.drop('Income_Category', axis=1, inplace=True)
eli_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br
0,Alameda,491.25,561.25,701.25,813.75,926.25
5,Alpine,408.75,467.50,583.75,677.50,771.25
10,Amador,363.75,415.00,518.75,602.50,685.00
15,Butte,308.75,352.50,440.00,511.25,581.25
20,Calaveras,366.25,418.75,522.50,606.25,690.00
25,Colusa,303.75,347.50,433.75,503.75,573.75
30,Contra Costa,491.25,561.25,701.25,813.75,926.25
35,Del Norte,303.75,347.50,433.75,503.75,573.75
40,El Dorado,400.00,457.50,571.25,663.75,755.00
45,Fresno,303.75,347.50,433.75,503.75,573.75


In [10]:
vli_df = df[df.Income_Category =="Very Low Income"].copy()
vli_df["1"]=vli_df["1"]*0.025
vli_df["2"]=vli_df["2"]*0.025
vli_df["4"]=vli_df["4"]*0.025
vli_df["6"]=vli_df["6"]*0.025
vli_df["8"]=vli_df["8"]*0.025
vli_df.rename(columns={"1":"VLI_studio","2":"VLI_1_br","4":"VLI_2_br","6":"VLI_3_br","8":"VLI_4_br"}, inplace=True)
vli_df.drop('Income_Category', axis=1, inplace=True)
vli_df

,County,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br
1,Alameda,818.75,935.00,1168.75,1356.25,1543.75
6,Alpine,681.25,778.75,972.50,1128.75,1283.75
11,Amador,606.25,692.50,865.00,1003.75,1142.50
16,Butte,513.75,587.50,733.75,851.25,968.75
21,Calaveras,610.00,697.50,871.25,1011.25,1151.25
26,Colusa,507.50,580.00,723.75,840.00,956.25
31,Contra Costa,818.75,935.00,1168.75,1356.25,1543.75
36,Del Norte,507.50,580.00,723.75,840.00,956.25
41,El Dorado,666.25,761.25,951.25,1103.75,1256.25
46,Fresno,507.50,580.00,723.75,840.00,956.25


In [11]:
li_df = df[df.Income_Category =="Lower Income"].copy()
li_df["1"]=li_df["1"]*0.025
li_df["2"]=li_df["2"]*0.025
li_df["4"]=li_df["4"]*0.025
li_df["6"]=li_df["6"]*0.025
li_df["8"]=li_df["8"]*0.025
li_df.rename(columns={"1":"LI_studio","2":"LI_1_br","4":"LI_2_br","6":"LI_3_br","8":"LI_4_br"}, inplace=True)
li_df.drop('Income_Category', axis=1, inplace=True)
li_df

,County,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br
2,Alameda,1158.75,1325.00,1656.25,1921.25,2186.25
7,Alpine,1090.00,1245.00,1556.25,1806.25,2055.00
12,Amador,968.75,1107.50,1383.75,1606.25,1827.50
17,Butte,822.50,940.00,1173.75,1362.50,1550.00
22,Calaveras,976.25,1115.00,1393.75,1617.50,1840.00
27,Colusa,811.25,926.25,1157.50,1343.75,1528.75
32,Contra Costa,1158.75,1325.00,1656.25,1921.25,2186.25
37,Del Norte,811.25,926.25,1157.50,1343.75,1528.75
42,El Dorado,1066.25,1218.75,1522.50,1766.25,2010.00
47,Fresno,811.25,926.25,1157.50,1343.75,1528.75


In [12]:
mi_df = df[df.Income_Category =="Median Income"].copy()
mi_df["1"]=mi_df["1"]*0.025
mi_df["2"]=mi_df["2"]*0.025
mi_df["4"]=mi_df["4"]*0.025
mi_df["6"]=mi_df["6"]*0.025
mi_df["8"]=mi_df["8"]*0.025
mi_df.rename(columns={"1":"MI_studio","2":"MI_1_br","4":"MI_2_br","6":"MI_3_br","8":"MI_4_br"}, inplace=True)
mi_df.drop('Income_Category', axis=1, inplace=True)
mi_df

,County,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br
3,Alameda,1636.25,1870.00,2337.50,2711.25,3085.00
8,Alpine,1492.50,1706.25,2132.50,2473.75,2815.00
13,Amador,1211.25,1383.75,1730.00,2006.25,2283.75
18,Butte,1027.50,1173.75,1467.50,1702.50,1937.50
23,Calaveras,1220.00,1393.75,1742.50,2021.25,2300.00
28,Colusa,1013.75,1157.50,1447.50,1678.75,1911.25
33,Contra Costa,1636.25,1870.00,2337.50,2711.25,3085.00
38,Del Norte,1013.75,1157.50,1447.50,1678.75,1911.25
43,El Dorado,1331.25,1522.50,1902.50,2207.50,2511.25
48,Fresno,1013.75,1157.50,1447.50,1678.75,1911.25


In [13]:
moi_df = df[df.Income_Category =="Moderate Income"].copy()
moi_df["1"]=moi_df["1"]*0.025
moi_df["2"]=moi_df["2"]*0.025
moi_df["4"]=moi_df["4"]*0.025
moi_df["6"]=moi_df["6"]*0.025
moi_df["8"]=moi_df["8"]*0.025
moi_df.rename(columns={"1":"MoI_studio","2":"MoI_1_br","4":"MoI_2_br","6":"MoI_3_br","8":"MoI_4_br"}, inplace=True)
moi_df.drop('Income_Category', axis=1, inplace=True)
moi_df

,County,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
4,Alameda,1963.75,2243.75,2805.00,3253.75,3702.50
9,Alpine,1791.25,2047.50,2558.75,2968.75,3377.50
14,Amador,1453.75,1661.25,2076.25,2408.75,2741.25
19,Butte,1232.50,1408.75,1761.25,2042.50,2325.00
24,Calaveras,1463.75,1672.50,2091.25,2426.25,2760.00
29,Colusa,1216.25,1390.00,1737.50,2015.00,2293.75
34,Contra Costa,1963.75,2243.75,2805.00,3253.75,3702.50
39,Del Norte,1216.25,1390.00,1737.50,2015.00,2293.75
44,El Dorado,1597.50,1826.25,2282.50,2647.50,3012.50
49,Fresno,1216.25,1390.00,1737.50,2015.00,2293.75


In [14]:
#combine dfs
all_amis_df=eli_df.merge(vli_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(li_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(mi_df, how='left', left_on = "County",right_on = "County")
all_amis_df=all_amis_df.merge(moi_df, how='left', left_on = "County",right_on = "County")
all_amis_df

,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,Alameda,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1158.75,1325.00,1656.25,1921.25,2186.25,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50
1,Alpine,408.75,467.50,583.75,677.50,771.25,681.25,778.75,972.50,1128.75,1283.75,1090.00,1245.00,1556.25,1806.25,2055.00,1492.50,1706.25,2132.50,2473.75,2815.00,1791.25,2047.50,2558.75,2968.75,3377.50
2,Amador,363.75,415.00,518.75,602.50,685.00,606.25,692.50,865.00,1003.75,1142.50,968.75,1107.50,1383.75,1606.25,1827.50,1211.25,1383.75,1730.00,2006.25,2283.75,1453.75,1661.25,2076.25,2408.75,2741.25
3,Butte,308.75,352.50,440.00,511.25,581.25,513.75,587.50,733.75,851.25,968.75,822.50,940.00,1173.75,1362.50,1550.00,1027.50,1173.75,1467.50,1702.50,1937.50,1232.50,1408.75,1761.25,2042.50,2325.00
4,Calaveras,366.25,418.75,522.50,606.25,690.00,610.00,697.50,871.25,1011.25,1151.25,976.25,1115.00,1393.75,1617.50,1840.00,1220.00,1393.75,1742.50,2021.25,2300.00,1463.75,1672.50,2091.25,2426.25,2760.00
5,Colusa,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
6,Contra Costa,491.25,561.25,701.25,813.75,926.25,818.75,935.00,1168.75,1356.25,1543.75,1158.75,1325.00,1656.25,1921.25,2186.25,1636.25,1870.00,2337.50,2711.25,3085.00,1963.75,2243.75,2805.00,3253.75,3702.50
7,Del Norte,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
8,El Dorado,400.00,457.50,571.25,663.75,755.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50
9,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75


## Part A4. Add count dummies for each unit based on income and unit type

In [15]:
# merge datasets
units_df=puma_county_df.merge(all_amis_df, how='left', left_on = "County",right_on = "County")
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br
0,H,7301,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25
1,H,3702,38.00,3.00,2.00,nan,3500.00,nan,nan,1.00,nan,600000.00,nan,nan,464800.00,6.00,64.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25
2,H,8503,78.00,2.00,6.00,430.00,1800.00,nan,460.00,1.00,nan,370000.00,nan,nan,16000.00,2.00,52.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50
3,H,300,233.00,1.00,5.00,nan,nan,680.00,nan,3.00,nan,nan,780.00,19.00,50000.00,3.00,nan,Alpine CA,Amador,Calaveras,Inyo,Mariposa,Mono,Tuolumne,Alpine,408.75,467.50,583.75,677.50,771.25,681.25,778.75,972.50,1128.75,1283.75,1090.00,1245.00,1556.25,1806.25,2055.00,1492.50,1706.25,2132.50,2473.75,2815.00,1791.25,2047.50,2558.75,2968.75,3377.50
4,H,7901,446.00,2.00,1.00,nan,nan,nan,nan,nan,3.00,140000.00,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,396.25,452.50,565.00,656.25,746.25,660.00,755.00,942.50,1093.75,1245.00,1056.25,1206.25,1507.50,1748.75,1990.00,1320.00,1507.50,1885.00,2186.25,2488.75,1583.75,1810.00,2262.50,2625.00,2986.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153584,H,3725,154.00,2.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1210.00,101.00,6000.00,5.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25
153585,H,7102,67.00,4.00,2.00,nan,1600.00,nan,600.00,1.00,nan,122000.00,nan,nan,22200.00,4.00,29.00,San Bernardino CA,NaN,NaN,NaN,NaN,NaN,NaN,San Bernardino,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.50,972.50,1106.25,938.75,1072.50,1340.00,1555.00,1770.00,1137.50,1300.00,1625.00,1885.00,2145.00,1365.00,1560.00,1950.00,2262.50,2573.75
153586,H,10701,104.00,3.00,2.00,nan,nan,580.00,nan,3.00,nan,nan,640.00,10.00,74500.00,5.00,nan,Tulare CA,NaN,NaN,NaN,NaN,NaN,NaN,Tulare,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75
153587,H,5903,357.00,4.00,2.00,nan,nan,nan,nan,nan,5.00,nan,nan,nan,nan,nan,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,953.75,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75


In [16]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['monthly_rent']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['monthly_rent']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['monthly_rent']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['monthly_rent']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['monthly_rent']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['rent_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['rent_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['rent_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['rent_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['rent_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['rent_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['rent_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['rent_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['rent_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['rent_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['rent_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['rent_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['rent_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['rent_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['rent_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['rent_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['rent_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['rent_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['rent_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['rent_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['rent_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['rent_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['rent_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['rent_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['rent_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['rent_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['rent_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['rent_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['rent_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['rent_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count
0,H,7301,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,3702,38.00,3.00,2.00,nan,3500.00,nan,nan,1.00,nan,600000.00,nan,nan,464800.00,6.00,64.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,8503,78.00,2.00,6.00,430.00,1800.00,nan,460.00,1.00,nan,370000.00,nan,nan,16000.00,2.00,52.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,H,300,233.00,1.00,5.00,nan,nan,680.00,nan,3.00,nan,nan,780.00,19.00,50000.00,3.00,nan,Alpine CA,Amador,Calaveras,Inyo,Mariposa,Mono,Tuolumne,Alpine,408.75,467.50,583.75,677.50,771.25,681.25,778.75,972.50,1128.75,1283.75,1090.00,1245.00,1556.25,1806.25,2055.00,1492.50,1706.25,2132.50,2473.75,2815.00,1791.25,2047.50,2558.75,2968.75,3377.50,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,H,7901,446.00,2.00,1.00,nan,nan,nan,nan,nan,3.00,140000.00,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,396.25,452.50,565.00,656.25,746.25,660.00,755.00,942.50,1093.75,1245.00,1056.25,1206.25,1507.50,1748.75,1990.00,1320.00,1507.50,1885.00,2186.25,2488.75,1583.75,1810.00,2262.50,2625.00,2986.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153584,H,3725,154.00,2.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1210.00,101.00,6000.00,5.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
153585,H,7102,67.00,4.00,2.00,nan,1600.00,nan,600.00,1.00,nan,122000.00,nan,nan,22200.00,4.00,29.00,San Bernardino CA,NaN,NaN,NaN,NaN,NaN,NaN,San Bernardino,352.50,402.50,502.50,583.75,663.75,586.25,670.00,837.5

## Part B1. Calculate total monthly housing cost by adding monthly first mortgage, second/junior mortgage, and condo fees

In [17]:
units_df['own_monthly_cost']=units_df['first_mortgage'] + units_df['second_mortgage'] + units_df['condo_fee'] + (units_df['prop_tax']/12)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost
0,H,7301,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
1,H,3702,38.00,3.00,2.00,nan,3500.00,nan,nan,1.00,nan,600000.00,nan,nan,464800.00,6.00,64.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
2,H,8503,78.00,2.00,6.00,430.00,1800.00,nan,460.00,1.00,nan,370000.00,nan,nan,16000.00,2.00,52.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2694.33
3,H,300,233.00,1.00,5.00,nan,nan,680.00,nan,3.00,nan,nan,780.00,19.00,50000.00,3.00,nan,Alpine CA,Amador,Calaveras,Inyo,Mariposa,Mono,Tuolumne,Alpine,408.75,467.50,583.75,677.50,771.25,681.25,778.75,972.50,1128.75,1283.75,1090.00,1245.00,1556.25,1806.25,2055.00,1492.50,1706.25,2132.50,2473.75,2815.00,1791.25,2047.50,2558.75,2968.75,3377.50,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,nan
4,H,7901,446.00,2.00,1.00,nan,nan,nan,nan,nan,3.00,140000.00,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,396.25,452.50,565.00,656.25,746.25,660.00,755.00,942.50,1093.75,1245.00,1056.25,1206.25,1507.50,1748.75,1990.00,1320.00,1507.50,1885.00,2186.25,2488.75,1583.75,1810.00,2262.50,2625.00,2986.25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153584,H,3725,154.00,2.00,8.00,nan,nan,1100.00,nan,3.00,nan,nan,1210.00,101.00,6000.00,5.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,nan
153585,H,7102,67.00,4.00,2.00,nan,1600.00,nan,600.00,1.00,nan,122000.00,nan,nan,22200.00,4.00,29.00,San Bernardino CA,NaN,NaN,NaN,NaN,NaN,NaN,San Bernardino,352.5

# Part B2. Add count dummies for each unit based on income and unit type

In [18]:
# create filter masks for each affordability and unit type
ELI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["ELI_studio"])
ELI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["ELI_1_br"])
ELI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["ELI_2_br"])
ELI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["ELI_3_br"])
ELI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["ELI_4_br"])
VLI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["VLI_studio"])
VLI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["VLI_1_br"])
VLI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["VLI_2_br"])
VLI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["VLI_3_br"])
VLI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["VLI_4_br"])
LI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["LI_studio"])
LI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["LI_1_br"])
LI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["LI_2_br"])
LI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["LI_3_br"])
LI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["LI_4_br"])
MI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MI_studio"])
MI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MI_1_br"])
MI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MI_2_br"])
MI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MI_3_br"])
MI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MI_4_br"])
MoI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']<=units_df["MoI_studio"])
MoI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']<=units_df["MoI_1_br"])
MoI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']<=units_df["MoI_2_br"])
MoI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']<=units_df["MoI_3_br"])
MoI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']<=units_df["MoI_4_br"])
HI_studio_mask=(units_df['number_bedrooms']==0)&(units_df['own_monthly_cost']>=units_df["MoI_studio"])
HI_1_br_mask=(units_df['number_bedrooms']==1)&(units_df['own_monthly_cost']>=units_df["MoI_1_br"])
HI_2_br_mask=(units_df['number_bedrooms']==2)&(units_df['own_monthly_cost']>=units_df["MoI_2_br"])
HI_3_br_mask=(units_df['number_bedrooms']==3)&(units_df['own_monthly_cost']>=units_df["MoI_3_br"])
HI_4_br_mask=(units_df['number_bedrooms']==4)&(units_df['own_monthly_cost']>=units_df["MoI_4_br"])

# apply filter masks to create dummy variables
units_df['own_ELI_studio_count']=np.where(ELI_studio_mask,1,0)
units_df['own_ELI_1_br_count']=np.where(ELI_1_br_mask,1,0)
units_df['own_ELI_2_br_count']=np.where(ELI_2_br_mask,1,0)
units_df['own_ELI_3_br_count']=np.where(ELI_3_br_mask,1,0)
units_df['own_ELI_4_br_count']=np.where(ELI_4_br_mask,1,0)
units_df['own_VLI_studio_count']=np.where(VLI_studio_mask,1,0)
units_df['own_VLI_1_br_count']=np.where(VLI_1_br_mask,1,0)
units_df['own_VLI_2_br_count']=np.where(VLI_2_br_mask,1,0)
units_df['own_VLI_3_br_count']=np.where(VLI_3_br_mask,1,0)
units_df['own_VLI_4_br_count']=np.where(VLI_4_br_mask,1,0)
units_df['own_LI_studio_count']=np.where(LI_studio_mask,1,0)
units_df['own_LI_1_br_count']=np.where(LI_1_br_mask,1,0)
units_df['own_LI_2_br_count']=np.where(LI_2_br_mask,1,0)
units_df['own_LI_3_br_count']=np.where(LI_3_br_mask,1,0)
units_df['own_LI_4_br_count']=np.where(LI_4_br_mask,1,0)
units_df['own_MI_studio_count']=np.where(MI_studio_mask,1,0)
units_df['own_MI_1_br_count']=np.where(MI_1_br_mask,1,0)
units_df['own_MI_2_br_count']=np.where(MI_2_br_mask,1,0)
units_df['own_MI_3_br_count']=np.where(MI_3_br_mask,1,0)
units_df['own_MI_4_br_count']=np.where(MI_4_br_mask,1,0)
units_df['own_MoI_studio_count']=np.where(MoI_studio_mask,1,0)
units_df['own_MoI_1_br_count']=np.where(MoI_1_br_mask,1,0)
units_df['own_MoI_2_br_count']=np.where(MoI_2_br_mask,1,0)
units_df['own_MoI_3_br_count']=np.where(MoI_3_br_mask,1,0)
units_df['own_MoI_4_br_count']=np.where(MoI_4_br_mask,1,0)
units_df['own_HI_studio_count']=np.where(HI_studio_mask,1,0)
units_df['own_HI_1_br_count']=np.where(HI_1_br_mask,1,0)
units_df['own_HI_2_br_count']=np.where(HI_2_br_mask,1,0)
units_df['own_HI_3_br_count']=np.where(HI_3_br_mask,1,0)
units_df['own_HI_4_br_count']=np.where(HI_4_br_mask,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,7301,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,H,3702,38.00,3.00,2.00,nan,3500.00,nan,nan,1.00,nan,600000.00,nan,nan,464800.00,6.00,64.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,H,8503,78.00,2.00,6.00,430.00,1800.00,nan,460.00,1.00,nan,370000.00,nan,nan,16000.00,2.00,52.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2694.33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,H,300,233.00,1.00,5.00,nan,nan,680.00,nan,3.00,nan,nan,780.00,19.00,50000.00,3.00,nan,Alpine CA,Amador,Calaveras,Inyo,Mariposa,Mono,Tuolumne,Alpine,408.75,467.50,583.75,677.50,771.25,681.25,778.75,972.50,1128.75,1283.75,1090.00,1245.00,1556.25,1806.25,2055.00,1492.50,1706.25,2132.50,2473.75,2815.00,1791.25,2047.50,2558.75,2968.75,3377.50,...,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,H,7901,446.00,2.00,1.00,nan,nan,nan,nan,nan,3.00,140000.00,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,396.25,452.50,565.00,656.25,746.25,660.00,755.00,942.50,1093.75,1245.00,1056.25,1206.25,1507.50,1748.75,1990.00,1320.00,1507.50,1885.00,2186.25,2488.75,1583.75,1810.00,2262.50,2625.00,2986.25,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

# Part C1. Multiply each dummy by the the household weight variable

In [19]:
units_df.rent_ELI_studio_count=units_df.rent_ELI_studio_count*units_df.weight
units_df.rent_ELI_1_br_count=units_df.rent_ELI_1_br_count*units_df.weight
units_df.rent_ELI_2_br_count=units_df.rent_ELI_2_br_count*units_df.weight
units_df.rent_ELI_3_br_count=units_df.rent_ELI_3_br_count*units_df.weight
units_df.rent_ELI_4_br_count=units_df.rent_ELI_4_br_count*units_df.weight
units_df.rent_VLI_studio_count=units_df.rent_VLI_studio_count*units_df.weight
units_df.rent_VLI_1_br_count=units_df.rent_VLI_1_br_count*units_df.weight
units_df.rent_VLI_2_br_count=units_df.rent_VLI_2_br_count*units_df.weight
units_df.rent_VLI_3_br_count=units_df.rent_VLI_3_br_count*units_df.weight
units_df.rent_VLI_4_br_count=units_df.rent_VLI_4_br_count*units_df.weight
units_df.rent_LI_studio_count=units_df.rent_LI_studio_count*units_df.weight
units_df.rent_LI_1_br_count=units_df.rent_LI_1_br_count*units_df.weight
units_df.rent_LI_2_br_count=units_df.rent_LI_2_br_count*units_df.weight
units_df.rent_LI_3_br_count=units_df.rent_LI_3_br_count*units_df.weight
units_df.rent_LI_4_br_count=units_df.rent_LI_4_br_count*units_df.weight
units_df.rent_MI_studio_count=units_df.rent_MI_studio_count*units_df.weight
units_df.rent_MI_1_br_count=units_df.rent_MI_1_br_count*units_df.weight
units_df.rent_MI_2_br_count=units_df.rent_MI_2_br_count*units_df.weight
units_df.rent_MI_3_br_count=units_df.rent_MI_3_br_count*units_df.weight
units_df.rent_MI_4_br_count=units_df.rent_MI_4_br_count*units_df.weight
units_df.rent_MoI_studio_count=units_df.rent_MoI_studio_count*units_df.weight
units_df.rent_MoI_1_br_count=units_df.rent_MoI_1_br_count*units_df.weight
units_df.rent_MoI_2_br_count=units_df.rent_MoI_2_br_count*units_df.weight
units_df.rent_MoI_3_br_count=units_df.rent_MoI_3_br_count*units_df.weight
units_df.rent_MoI_4_br_count=units_df.rent_MoI_4_br_count*units_df.weight
units_df.rent_HI_studio_count=units_df.rent_HI_studio_count*units_df.weight
units_df.rent_HI_1_br_count=units_df.rent_HI_1_br_count*units_df.weight
units_df.rent_HI_2_br_count=units_df.rent_HI_2_br_count*units_df.weight
units_df.rent_HI_3_br_count=units_df.rent_HI_3_br_count*units_df.weight
units_df.rent_HI_4_br_count=units_df.rent_HI_4_br_count*units_df.weight
units_df.own_ELI_studio_count=units_df.own_ELI_studio_count*units_df.weight
units_df.own_ELI_1_br_count=units_df.own_ELI_1_br_count*units_df.weight
units_df.own_ELI_2_br_count=units_df.own_ELI_2_br_count*units_df.weight
units_df.own_ELI_3_br_count=units_df.own_ELI_3_br_count*units_df.weight
units_df.own_ELI_4_br_count=units_df.own_ELI_4_br_count*units_df.weight
units_df.own_VLI_studio_count=units_df.own_VLI_studio_count*units_df.weight
units_df.own_VLI_1_br_count=units_df.own_VLI_1_br_count*units_df.weight
units_df.own_VLI_2_br_count=units_df.own_VLI_2_br_count*units_df.weight
units_df.own_VLI_3_br_count=units_df.own_VLI_3_br_count*units_df.weight
units_df.own_VLI_4_br_count=units_df.own_VLI_4_br_count*units_df.weight
units_df.own_LI_studio_count=units_df.own_LI_studio_count*units_df.weight
units_df.own_LI_1_br_count=units_df.own_LI_1_br_count*units_df.weight
units_df.own_LI_2_br_count=units_df.own_LI_2_br_count*units_df.weight
units_df.own_LI_3_br_count=units_df.own_LI_3_br_count*units_df.weight
units_df.own_LI_4_br_count=units_df.own_LI_4_br_count*units_df.weight
units_df.own_MI_studio_count=units_df.own_MI_studio_count*units_df.weight
units_df.own_MI_1_br_count=units_df.own_MI_1_br_count*units_df.weight
units_df.own_MI_2_br_count=units_df.own_MI_2_br_count*units_df.weight
units_df.own_MI_3_br_count=units_df.own_MI_3_br_count*units_df.weight
units_df.own_MI_4_br_count=units_df.own_MI_4_br_count*units_df.weight
units_df.own_MoI_studio_count=units_df.own_MoI_studio_count*units_df.weight
units_df.own_MoI_1_br_count=units_df.own_MoI_1_br_count*units_df.weight
units_df.own_MoI_2_br_count=units_df.own_MoI_2_br_count*units_df.weight
units_df.own_MoI_3_br_count=units_df.own_MoI_3_br_count*units_df.weight
units_df.own_MoI_4_br_count=units_df.own_MoI_4_br_count*units_df.weight
units_df.own_HI_studio_count=units_df.own_HI_studio_count*units_df.weight
units_df.own_HI_1_br_count=units_df.own_HI_1_br_count*units_df.weight
units_df.own_HI_2_br_count=units_df.own_HI_2_br_count*units_df.weight
units_df.own_HI_3_br_count=units_df.own_HI_3_br_count*units_df.weight
units_df.own_HI_4_br_count=units_df.own_HI_4_br_count*units_df.weight
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
0,H,7301,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,H,3702,38.00,3.00,2.00,nan,3500.00,nan,nan,1.00,nan,600000.00,nan,nan,464800.00,6.00,64.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,H,8503,78.00,2.00,6.00,430.00,1800.00,nan,460.00,1.00,nan,370000.00,nan,nan,16000.00,2.00,52.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2694.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,78.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,H,300,233.00,1.00,5.00,nan,nan,680.00,nan,3.00,nan,nan,780.00,19.00,50000.00,3.00,nan,Alpine CA,Amador,Calaveras,Inyo,Mariposa,Mono,Tuolumne,Alpine,408.75,467.50,583.75,677.50,771.25,681.25,778.75,972.50,1128.75,1283.75,1090.00,1245.00,1556.25,1806.25,2055.00,1492.50,1706.25,2132.50,2473.75,2815.00,1791.25,2047.50,2558.75,2968.75,3377.50,...,233.00,0.00,0.00,0.00,0.00,233.00,0.00,0.00,0.00,0.00,233.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,H,7901,446.00,2.00,1.00,nan,nan,nan,nan,nan,3.00,140000.00,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,396.25,452.50,565.00,656.25,746.25,660.00

In [20]:
units_df[units_df.rent_HI_4_br_count>0]

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
51,H,5901,77.00,4.00,2.00,nan,nan,3600.00,nan,3.00,nan,nan,3743.00,18.00,250000.00,2.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,953.75,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,77.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2421,H,3718,44.00,4.00,2.00,nan,nan,3600.00,nan,3.00,nan,nan,3850.00,9.00,521000.00,3.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,44.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3518,H,5301,36.00,4.00,2.00,nan,nan,2900.00,nan,3.00,nan,nan,2900.00,31.00,112400.00,1.00,nan,Monterey CA,NaN,NaN,NaN,NaN,NaN,NaN,Monterey,375.00,428.75,535.00,621.25,706.25,625.00,713.75,891.25,1035.00,1177.50,998.75,1141.25,1426.25,1655.00,1883.75,1202.50,1373.75,1717.50,1992.50,2267.50,1442.50,1648.75,2061.25,2391.25,2721.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,36.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5150,H,5901,104.00,4.00,2.00,nan,nan,3600.00,nan,3.00,nan,nan,3745.00,16.00,282000.00,1.00,nan,Orange CA,NaN,NaN,NaN,NaN,NaN,NaN,Orange,506.25,578.75,722.50,838.75,953.75,843.75,963.75,1203.75,1397.50,1590.00,1348.75,1541.25,1926.25,2235.00,2543.75,1526.25,1743.75,2180.00,2528.75,2877.50,1831.25,2092.50,2616.25,3035.00,3453.75,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,104.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5626,H,3726,195.00,4.00,2.00,nan,nan,3600.00,nan,3.00,nan,nan,4110.00,34.00,144000.00,4.00,nan,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,106

# Part D1. Add dummy variables for vacant "for-rent," vacant "for-sale," and recently moved in

In [21]:
# create dummy variables for vacant for rent, vacant for sale, rented in the last year, and sold in the last year
units_df["vacant_for_rent"]=np.where(units_df['vacancy_status']==1,1,0)
units_df["vacant_for_sale"]=np.where(units_df['vacancy_status']==3,1,0)
units_df["moved_last_year"]=np.where(units_df['moved_in']==1,1,0)
units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
0,H,7301,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
1,H,3702,38.00,3.00,2.00,nan,3500.00,nan,nan,1.00,nan,600000.00,nan,nan,464800.00,6.00,64.00,Los Angeles CA,NaN,NaN,NaN,NaN,NaN,NaN,Los Angeles,448.75,512.50,640.00,742.50,845.00,747.50,855.00,1067.50,1238.75,1410.00,1196.25,1366.25,1707.50,1981.25,2255.00,1133.75,1296.25,1620.00,1878.75,2138.75,1361.25,1555.00,1943.75,2255.00,2566.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
2,H,8503,78.00,2.00,6.00,430.00,1800.00,nan,460.00,1.00,nan,370000.00,nan,nan,16000.00,2.00,52.00,Santa Clara CA,NaN,NaN,NaN,NaN,NaN,NaN,Santa Clara,557.50,637.50,796.25,923.75,1051.25,928.75,1061.25,1326.25,1538.75,1751.25,1485.00,1697.50,2122.50,2461.25,2801.25,1846.25,2110.00,2637.50,3060.00,3481.25,2215.00,2532.50,3165.00,3671.25,4177.50,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2694.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,78.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
3,H,300,233.00,1.00,5.00,nan,nan,680.00,nan,3.00,nan,nan,780.00,19.00,50000.00,3.00,nan,Alpine CA,Amador,Calaveras,Inyo,Mariposa,Mono,Tuolumne,Alpine,408.75,467.50,583.75,677.50,771.25,681.25,778.75,972.50,1128.75,1283.75,1090.00,1245.00,1556.25,1806.25,2055.00,1492.50,1706.25,2132.50,2473.75,2815.00,1791.25,2047.50,2558.75,2968.75,3377.50,...,0.00,0.00,233.00,0.00,0.00,0.00,0.00,233.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0
4,H,7901,446.00,2.00,1.00,nan,nan,nan,nan,nan,3.00,140000.00,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,396.25,452.50,565.00,656.25,746.25,660.00,755.00,942.50,1093.75,1245.00,1056.25,1206.25,

In [22]:
#filter for only currently listed or recently moved into units
recent_mask = (units_df.vacant_for_rent==1)|(units_df.vacant_for_sale==1)|(units_df.moved_last_year==1)
recent_units_df = units_df[recent_mask]
recent_units_df

,record_type,PUMA,weight,number_bedrooms,units_in_structure,condo_fee,first_mortgage,monthly_rent,second_mortgage,tenure,vacancy_status,property_value,gross_rent,gross_rent_pct_of_income,hh_income,moved_in,prop_tax,county1,county2,county3,county4,county5,county6,county7,County,ELI_studio,ELI_1_br,ELI_2_br,ELI_3_br,ELI_4_br,VLI_studio,VLI_1_br,VLI_2_br,VLI_3_br,VLI_4_br,LI_studio,LI_1_br,LI_2_br,LI_3_br,LI_4_br,MI_studio,MI_1_br,MI_2_br,MI_3_br,MI_4_br,MoI_studio,MoI_1_br,MoI_2_br,MoI_3_br,MoI_4_br,...,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_monthly_cost,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count,vacant_for_rent,vacant_for_sale,moved_last_year
4,H,7901,446.00,2.00,1.00,nan,nan,nan,nan,nan,3.00,140000.00,nan,nan,nan,nan,nan,San Luis Obispo CA,NaN,NaN,NaN,NaN,NaN,NaN,San Luis Obispo,396.25,452.50,565.00,656.25,746.25,660.00,755.00,942.50,1093.75,1245.00,1056.25,1206.25,1507.50,1748.75,1990.00,1320.00,1507.50,1885.00,2186.25,2488.75,1583.75,1810.00,2262.50,2625.00,2986.25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,1,0
11,H,1904,78.00,2.00,9.00,nan,nan,570.00,nan,3.00,nan,nan,650.00,89.00,8810.00,1.00,nan,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1447.50,1678.75,1911.25,1216.25,1390.00,1737.50,2015.00,2293.75,...,0.00,0.00,0.00,78.00,0.00,0.00,0.00,0.00,78.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
22,H,6103,37.00,3.00,2.00,nan,nan,1200.00,nan,3.00,nan,nan,1527.00,46.00,40000.00,1.00,nan,Placer CA,NaN,NaN,NaN,NaN,NaN,NaN,Placer,400.00,457.50,571.25,663.75,755.00,666.25,761.25,951.25,1103.75,1256.25,1066.25,1218.75,1522.50,1766.25,2010.00,1331.25,1522.50,1902.50,2207.50,2511.25,1597.50,1826.25,2282.50,2647.50,3012.50,...,0.00,0.00,0.00,0.00,37.00,0.00,0.00,0.00,0.00,37.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
24,H,7302,64.00,3.00,2.00,nan,nan,1700.00,nan,3.00,nan,nan,1790.00,26.00,82600.00,1.00,nan,San Diego CA,NaN,NaN,NaN,NaN,NaN,NaN,San Diego,433.75,496.25,620.00,718.75,818.75,722.50,826.25,1032.50,1197.50,1362.50,1156.25,1322.50,1652.50,1917.50,2181.25,1328.75,1517.50,1897.50,2201.25,2505.00,1593.75,1822.50,2277.50,2642.50,3006.25,...,0.00,0.00,0.00,0.00,64.00,0.00,0.00,0.00,0.00,64.00,0.00,0.00,0.00,0.00,0.00,0.00,nan,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1
30,H,1904,49.00,5.00,3.00,nan,2500.00,nan,nan,1.00,nan,400000.00,nan,nan,333000.00,1.00,23.00,Fresno CA,NaN,NaN,NaN,NaN,NaN,NaN,Fresno,303.75,347.50,433.75,503.75,573.75,507.50,580.00,723.75,840.00,956.25,811.25,926.25,1157.50,1343.75,1528.75,1013.75,1157.50,1

# Part D2. Aggregating at the PUMA level

In [23]:
# for all units
puma_sums_df = units_df.groupby("PUMA").sum()
puma_counts_df = puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,628.00,1630.00,966.00,50.00,0.00,1185.00,4316.00,2456.00,452.00,0.00,2240.00,8795.00,5931.00,945.00,69.00,2912.00,12934.00,9401.00,1848.00,289.00,3138.00,14434.00,9729.00,2356.00,495.00,0.00,280.00,561.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,158.00,44.00,0.00,0.00,0.00,158.00,44.00,0.00,0.00,0.00,0.00,0.00,0.00
102,1784.00,3861.00,3002.00,2209.00,251.00,4710.00,11584.00,8652.00,3385.00,884.00,6363.00,19095.00,13589.00,5529.00,1029.00,7017.00,21549.00,16013.00,5861.00,1133.00,7377.00,22292.00,17090.00,5861.00,1133.00,268.00,247.00,395.00,249.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,113.00,0.00,0.00,0.00,214.00,113.00,0.00,0.00,0.00,282.00,113.00,0.00,0.00,58.00,65.00,0.00,50.00,0.00
103,596.00,593.00,462.00,534.00,79.00,1680.00,1780.00,1944.00,625.00,111.00,2229.00,5389.00,4513.00,1189.00,592.00,2517.00,7568.00,5917.00,1646.00,1049.00,2517.00,7568.00,6084.00,2165.00,1253.00,0.00,0.00,55.00,231.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00,59.00,137.00,61.00,0.00,0.00,0.00,65.00,204.00,87.00,0.00
104,187.00,360.00,1590.00,1113.00,132.00,894.00,3354.00,7773.00,3363.00,810.00,894.00,4617.00,10866.00,5573.00,1452.00,894.00,4653.00,10961.00,5573.00,1668.00,894.00,4653.00,10961.00,5573.00,1668.00,0.00,67.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
105,549.00,1503.00,774.00,618.00,42.00,1244.00,4222.00,4760.00,1305.00,109.00,1738.00,9521.00,11372.00,2864.00,220.00,2374.00,10161.00,13495.00,4497.00,699.00,2374.00,10161.00,13559.00,4607.00,870.00,126.00,177.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72.00,0.00,0.00,0.00,0.00,72.00,72.00,110.00,0.00,0.00,72.00,72.00,110.00,0.00,0.00,0.00,72.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,225.00,1558.00,33.00,689.00,38.00,679.00,2581.00,2855.00,1318.00,510.00,1155.00,4291.00,8818.00,5140.00,2356.00,1155.00,4483.00,9676.00,5504.00,2651.00,1155.00,4483.00,9799.00,5580.00,2651.00,0.00,0.00,0.00,0.00,135.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,35.00,28.00,0.00,0.00,0.00,35.00,28.00,0.00,0.00,0.00,35.00,28.00,0.00,0.00,0.00,49.00,0.00,0.00
11104,711.00,1188.00,638.00,572.00,141.00,1085.00,1879.00,1856.00,1067.00,171.00,1213.00,6473.00,5778.00,3232.00,1193.00,1213.00,6853.00,6156.00,4144.00,1533.00,1272.00,7244.00,6516.00,4144.00,1719.00,665.00,377.00,85

In [24]:
#export it
puma_counts_df.to_csv("weighted_puma_counts.csv")

In [25]:
# for recent units
recent_puma_sums_df = recent_units_df.groupby("PUMA").sum()
recent_puma_counts_df = recent_puma_sums_df[["rent_ELI_studio_count","rent_ELI_1_br_count",
    "rent_ELI_2_br_count",
    "rent_ELI_3_br_count",
    "rent_ELI_4_br_count",
    "rent_VLI_studio_count",
    "rent_VLI_1_br_count",
    "rent_VLI_2_br_count",
    "rent_VLI_3_br_count",
    "rent_VLI_4_br_count",
    "rent_LI_studio_count",
    "rent_LI_1_br_count",
    "rent_LI_2_br_count",
    "rent_LI_3_br_count",
    "rent_LI_4_br_count",
    "rent_MI_studio_count",
    "rent_MI_1_br_count",
    "rent_MI_2_br_count",
    "rent_MI_3_br_count",
    "rent_MI_4_br_count",
    "rent_MoI_studio_count",
    "rent_MoI_1_br_count",
    "rent_MoI_2_br_count",
    "rent_MoI_3_br_count",
    "rent_MoI_4_br_count",
    "rent_HI_studio_count",
    "rent_HI_1_br_count",
    "rent_HI_2_br_count",
    "rent_HI_3_br_count",
    "rent_HI_4_br_count",
    "own_ELI_studio_count",
    "own_ELI_1_br_count",
    "own_ELI_2_br_count",
    "own_ELI_3_br_count",
    "own_ELI_4_br_count",
    "own_VLI_studio_count",
    "own_VLI_1_br_count",
    "own_VLI_2_br_count",
    "own_VLI_3_br_count",
    "own_VLI_4_br_count",
    "own_LI_studio_count",
    "own_LI_1_br_count",
    "own_LI_2_br_count",
    "own_LI_3_br_count",
    "own_LI_4_br_count",
    "own_MI_studio_count",
    "own_MI_1_br_count",
    "own_MI_2_br_count",
    "own_MI_3_br_count",
    "own_MI_4_br_count",
    "own_MoI_studio_count",
    "own_MoI_1_br_count",
    "own_MoI_2_br_count",
    "own_MoI_3_br_count",
    "own_MoI_4_br_count",
    "own_HI_studio_count",
    "own_HI_1_br_count",
    "own_HI_2_br_count",
    "own_HI_3_br_count",
    "own_HI_4_br_count"]].copy()
recent_puma_counts_df

,rent_ELI_studio_count,rent_ELI_1_br_count,rent_ELI_2_br_count,rent_ELI_3_br_count,rent_ELI_4_br_count,rent_VLI_studio_count,rent_VLI_1_br_count,rent_VLI_2_br_count,rent_VLI_3_br_count,rent_VLI_4_br_count,rent_LI_studio_count,rent_LI_1_br_count,rent_LI_2_br_count,rent_LI_3_br_count,rent_LI_4_br_count,rent_MI_studio_count,rent_MI_1_br_count,rent_MI_2_br_count,rent_MI_3_br_count,rent_MI_4_br_count,rent_MoI_studio_count,rent_MoI_1_br_count,rent_MoI_2_br_count,rent_MoI_3_br_count,rent_MoI_4_br_count,rent_HI_studio_count,rent_HI_1_br_count,rent_HI_2_br_count,rent_HI_3_br_count,rent_HI_4_br_count,own_ELI_studio_count,own_ELI_1_br_count,own_ELI_2_br_count,own_ELI_3_br_count,own_ELI_4_br_count,own_VLI_studio_count,own_VLI_1_br_count,own_VLI_2_br_count,own_VLI_3_br_count,own_VLI_4_br_count,own_LI_studio_count,own_LI_1_br_count,own_LI_2_br_count,own_LI_3_br_count,own_LI_4_br_count,own_MI_studio_count,own_MI_1_br_count,own_MI_2_br_count,own_MI_3_br_count,own_MI_4_br_count,own_MoI_studio_count,own_MoI_1_br_count,own_MoI_2_br_count,own_MoI_3_br_count,own_MoI_4_br_count,own_HI_studio_count,own_HI_1_br_count,own_HI_2_br_count,own_HI_3_br_count,own_HI_4_br_count
PUMA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
101,0.00,255.00,94.00,0.00,0.00,261.00,337.00,559.00,0.00,0.00,943.00,1763.00,1916.00,114.00,0.00,1368.00,4206.00,3832.00,658.00,220.00,1368.00,5457.00,3971.00,721.00,361.00,0.00,196.00,87.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
102,861.00,626.00,853.00,724.00,51.00,1863.00,1920.00,1653.00,919.00,191.00,2320.00,4679.00,2825.00,2027.00,191.00,2624.00,5309.00,4109.00,2148.00,191.00,2624.00,5653.00,4713.00,2148.00,191.00,0.00,123.00,206.00,186.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
103,0.00,0.00,221.00,162.00,79.00,0.00,66.00,340.00,253.00,79.00,217.00,753.00,1430.00,358.00,254.00,217.00,1878.00,1828.00,621.00,457.00,217.00,1878.00,1928.00,1140.00,603.00,0.00,0.00,0.00,142.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
104,107.00,54.00,132.00,178.00,132.00,408.00,528.00,1909.00,719.00,456.00,408.00,977.00,2909.00,1127.00,589.00,408.00,1013.00,3004.00,1127.00,589.00,408.00,1013.00,3004.00,1127.00,589.00,0.00,67.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
105,288.00,447.00,0.00,131.00,0.00,645.00,549.00,442.00,131.00,0.00,747.00,1985.00,2190.00,498.00,0.00,1076.00,2258.00,3058.00,689.00,0.00,1076.00,2258.00,3122.00,689.00,71.00,126.00,117.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,0.00,0.00,0.00,108.00,0.00,0.00,180.00,0.00,182.00,292.00,110.00,1037.00,1647.00,1155.00,737.00,110.00,1229.00,1893.00,1309.00,737.00,110.00,1229.00,2016.00,1385.00,737.00,0.00,0.00,0.00,0.00,135.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11104,203.00,0.00,104.00,0.00,0.00,356.00,367.00,574.00,0.00,0.00,416.00,2184.00,2022.00,784.00,177.00,416.00,2437.00,2243.00,1112.00,177.00,416.00,2719.00,2388.00,1112.00,177.00,584.00,232.00,0.00,139.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
11105,0.00,0.0

In [26]:
#export it
recent_puma_counts_df.to_csv("weighted_recent_puma_counts.csv")